In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys 
sys.path.append('DGL_pipeline')
from sage_tabular_model import SAGE_tabular
import configparser
import torch
from data_utils import networkx_to_torch, normalize_features, replace_na, count_number_params
import networkx as nx
import dgl

Using backend: pytorch


In [3]:
import numpy as np

In [4]:
import configparser

In [5]:
from sklearn.model_selection import ParameterGrid

In [22]:
config_ = configparser.ConfigParser(allow_no_value=True)
config_.read('/home/Container/tgnn/scripts/experimental_sripts/hp_config.ini')
config = config_

In [23]:
config_option = 'NODE'

In [25]:
config.get(config_option, 'learning_rate')

In [87]:
layer_kwargs = eval(config[config_option]['layer_kwargs'])
tab_net_params = eval(config[config_option]['tab_net'])
graph_net_params = eval(config[config_option]['graph_net'])

In [91]:
tab_net_params['layer_kwargs'] = list(ParameterGrid(layer_kwargs))

In [92]:
grid = ParameterGrid(tab_net_params)

In [93]:
list(grid)

[{'cat_dims': 16,
  'dims': [128],
  'layer_kwargs': {'flatten_output': True, 'num_layers': 2, 'tree_dim': 3},
  'layer_type': 'Dense',
  'nclass': 128}]

In [9]:
_ = config['MGBDT']

configparser.SectionProxy

In [5]:
graph_path = '/home/Container/bgnn/datasets/house/graph.graphml'

In [6]:
networkx_graph = nx.read_graphml(graph_path) 
networkx_graph = nx.relabel_nodes(networkx_graph, {str(i): i for i in range(len(networkx_graph))}) 

graph = networkx_to_torch(networkx_graph)

In [24]:
sg = graph.subgraph([100, 200, 599])

In [25]:
sg.srcdata['feats'] = torch.Tensor([100, 200, 599])

In [26]:
sg

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'feats': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})

In [37]:
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
nids = [100]
dataloader = dgl.dataloading.NodeDataLoader(sg, [0,1,2], sampler, batch_size=128,
                                        shuffle=False,
                                        drop_last=False)

In [42]:
_,_,blocks = next(iter(dataloader))

In [43]:
block = blocks[0]

In [46]:
b = dgl.to_homogeneous(block)

In [50]:
block.ndata

{'_N': {'_ID': tensor([0, 1, 2]), 'feats': tensor([100., 200., 599.])}}

In [47]:
b.ndata['h'] = torch.Tensor([100, 200, 599])

DGLError: Expect number of features to match number of nodes (len(u)). Got 3 and 6 instead.

In [8]:
for smth in dataloader:
    print(smth)

(tensor([0, 1, 2]), tensor([0, 1, 2]), [Block(num_src_nodes=3, num_dst_nodes=3, num_edges=3)])


In [10]:
inp_nodes, _, blocks = next(iter(dataloader))

In [14]:
b = blocks[0]

In [20]:
inp_nodes.numpy()

array([0, 1, 2])

In [28]:
dataloader.dataloader.

[0, 1, 2]

In [26]:
sg.ndata[dgl.NID][inp_nodes]

tensor([100, 200, 599])

In [32]:
dataloader.collator.g.ndata

{'_ID': tensor([100, 200, 599])}

In [30]:
blocks[0].ndata

{'_N': {'_ID': tensor([0, 1, 2])}}